In [1]:
import mxnet as mx
    
from mxnet import gluon , nd , autograd

from mxnet.gluon import nn

import pandas as pd
import numpy as np
import datetime
import os

In [1]:
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA

from tkinter import filedialog
from tkinter import *

def browse_button():
    # Allow user to select a directory and store it in global var
    # called folder_path
    global folder_path
    filename = filedialog.askopenfile()
    folder_path.set(filename)
    print(filename)
    return filename


root = Tk()
folder_path = StringVar()
lbl1 = Label(master=root,textvariable=folder_path)
lbl1.grid(row=0, column=1)
button2 = Button(text="Browse", command=browse_button)
button2.grid(row=0, column=3)

x = browse_button()
mainloop()


<_io.TextIOWrapper name='D:/Python/Hackathon/Orderhealth/All_Base.xlsx' mode='r' encoding='cp1252'>


In [37]:
df = pd.read_excel(x.name)

In [62]:
df = pd.read_csv(r"D:\Python\Hackathon\Orderhealth\All-A.csv")

In [63]:
df.tail(10)

,Date,A
16291,12-08-2019,90
16292,13-08-2019,34
16293,14-08-2019,41
16294,15-08-2019,70
16295,16-08-2019,64
16296,17-08-2019,43
16297,18-08-2019,61
16298,19-08-2019,38
16299,20-08-2019,83
16300,21-08-2019,46


In [64]:
df = df[['A']]

In [65]:
df = df.dropna()

there is no data on the 1st of every month for field A&B

there is no data for field C&D on last day of every month

so we replaced those values with -1
if you want to change it according to you, please proceed

rest we need to find out the there may be dependencies of the fields on each other

rest every field is not dependent on holiday or something

In [66]:
def dayyear(x):
    
    
    #year = datetime.datetime.strptime(str(x) , "%Y-%m-%d").year
    #strval = datetime.datetime.strftime(x ,"%Y-%m-%d")
    
    year = datetime.datetime.strptime(x , "%d-%m-%Y").year
    month = datetime.datetime.strptime(x , "%d-%m-%Y").month
    day = datetime.datetime.strptime(x, "%d-%m-%Y").day

    
    return [year , month , day]

In [67]:
test=  df["Date"].apply(dayyear)
df2 = pd.DataFrame(test.tolist(), columns=['Year', 'Month' , "day"])
df = pd.concat([df , df2] , axis=1)

KeyError: 'Date'

In [68]:
df.head()

,A
0,39
1,49
2,52
3,19
4,1


In [69]:
del df['Date']

KeyError: 'Date'

In [70]:
df.Year = pd.Categorical(df.Year)

df.Month = pd.Categorical(df.Month)

df.day = pd.Categorical(df.day)

AttributeError: 'DataFrame' object has no attribute 'Year'

In [71]:
df.head()

,A
0,39
1,49
2,52
3,19
4,1


In [72]:
df.isna().sum()

A    0
dtype: int64

In [73]:
df.isin([-1]).sum()

A    0
dtype: int64

In [74]:
df_2 = df.replace({-1:0})

In [75]:
df_2.tail()

,A
16296,43
16297,61
16298,38
16299,83
16300,46


In [76]:
df_2.isin([-1]).sum()

A    0
dtype: int64

In [77]:
df_2.isna().sum()

A    0
dtype: int64

In [78]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16301 entries, 0 to 16300
Data columns (total 1 columns):
A    16301 non-null object
dtypes: object(1)
memory usage: 254.7+ KB


In [79]:
df_2 = df_2.replace({'##':np.nan})

df_2 = df_2.replace({"Null":np.nan})

df_2 = df_2.replace({"	Null":np.nan})

#df_2[['A']] = pd.to_numeric(df_2.A)

In [80]:
df_2[['A']] = pd.DataFrame(pd.to_numeric(df_2.A))

In [82]:
df2.isna().sum()

A    0
dtype: int64

In [83]:
df_2.isna().sum()

A    81
dtype: int64

In [85]:
df2 =df_2.fillna(0)

In [22]:
df2 = df_2.interpolate(method= 'linear' )

In [86]:
len(df2)

16301

In [87]:
df_train = df2
df_test =df2

In [88]:
# df_train = df2.iloc[0 : 1600 , :]



# df_test = df2.iloc[1600 : len(df) , :]




In [89]:
plt.plot(df.Output[10:100])

NameError: name 'plt' is not defined

In [90]:
df_train.tail()


,A
16296,43.0
16297,61.0
16298,38.0
16299,83.0
16300,46.0


In [91]:
lot_size = 1
mins = 0
maxs = 1

In [92]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = sc.fit_transform(np.array(df_train).reshape(-1,1))


#training_set_scaled = np.array(training_set_scaled)
training_set_scaled = np.array(df_train)
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(lot_size, len(df_train)):
    #X_train.append(training_set_scaled[i-lot_size:i, 4:7])
    X_train.append(training_set_scaled[i-lot_size:i, mins:maxs])
    y_train.append(training_set_scaled[i, mins:maxs])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], (maxs- mins)))

In [93]:
X_train.shape

(16300, 1, 1)

In [94]:
X_train

array([[[39.]],

       [[49.]],

       [[52.]],

       ...,

       [[61.]],

       [[38.]],

       [[83.]]])

In [95]:
y_train

array([[49.],
       [52.],
       [19.],
       ...,
       [38.],
       [83.],
       [46.]])

In [96]:
y_train.shape

(16300, 1)

In [97]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = sc.fit_transform(np.array(df_open).reshape(-1,1))


training_set_scaled = np.array(df_test)
#training_set_scaled = np.array(df_test)
# Creating a data structure with 60 timesteps and 1 output
X_test = []
y_test = []
for i in range(lot_size, len(df_test)):
    #X_train.append(training_set_scaled[i-lot_size:i, 4:7])
    X_test.append(training_set_scaled[i-lot_size:i, mins:maxs])
    y_test.append(training_set_scaled[i, mins:maxs])
X_test, y_test = np.array(X_test), np.array(y_test)

# Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], (maxs- mins)))

In [98]:
X_test.shape

(16300, 1, 1)

In [99]:
y_test.shape

(16300, 1)

In [100]:
context = mx.cpu()

In [101]:
def build_net(net):
    
    with net.name_scope():
            
        #net.add(mx.gluon.rnn.LSTM(5 , bidirectional = True ))
        net.add(mx.gluon.rnn.LSTM(3 , bidirectional = True ))

        net.add(mx.gluon.nn.Dense(328,activation='relu' ))
        net.add(mx.gluon.nn.Dense(164,activation='relu' ))
        net.add(mx.gluon.nn.Dense(64,activation='relu'))
        net.add(mx.gluon.nn.Dense(32,activation='relu'))
        net.add(mx.gluon.nn.Dense(1 ))
    
    
        return net
        
    
    
    

In [102]:
def build_net(net):
    
    with net.name_scope():
            
        #net.add(mx.gluon.rnn.LSTM(10 , bidirectional = True ))
        net.add(mx.gluon.rnn.LSTM(5 , bidirectional = True  ))
        net.add(mx.gluon.rnn.LSTM(3 , bidirectional = True ))
        net.add(mx.gluon.rnn.LSTM(1 , bidirectional = True ))
        #net.add(mx.gluon.nn.BatchNorm())
        
        net.add(mx.gluon.nn.Dense(656,activation='relu' ))
        
        
        net.add(mx.gluon.nn.Dense(328,activation='relu' ))
        
        
        net.add(mx.gluon.nn.Dense(164,activation='relu' ))
      
        net.add(mx.gluon.nn.Dense(82,activation='relu'))
        
        net.add(mx.gluon.nn.Dense(41,activation='relu'))

        
        net.add(mx.gluon.nn.Dense(1 ))
    
    
        return net
    
    
    #net.add(mx.gluon.nn.BatchNorm(axis=1, center=True, scale=True))  ,7
        

In [103]:
model = build_net(gluon.nn.Sequential())

In [104]:
def evaluate_accuracy(data_iterator , model , L):
    
    loss_avg = 0.
    for i , (data,label) in enumerate(data_iterator):
        
        data = data.as_in_context(context).reshape((-1 ,lot_size , (maxs - mins)))
        label = label.as_in_context(context).reshape((-1, 1, (maxs - mins)))
        output = model(data)
        
        loss = L(output , label)
        loss_avg = (loss_avg*i+nd.mean(loss).asscalar())/(i+1)
        
    return loss_avg

In [105]:
def dataprep(X):
    
    
    
    
    training_set_scaled = X
    # Creating a data structure with 60 timesteps and 1 output
    X_train = []
    y_train = []
    for i in range(lot_size, len(X)):
        #X_train.append(training_set_scaled[i-lot_size:i, 4:7])
        X_train.append(training_set_scaled[i-lot_size:i, mins:maxs])
        y_train.append(training_set_scaled[i, mins:maxs])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Reshaping
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], (maxs- mins)))
    
    print("Y>>>>>>>>>>>>>>>>>>")
    #print(y_train)
    
    print("X>>>>>>>>>>>>>>>>>>")
    #print(X_train)
#     print(X_train.shape)
    
#     print(type(X_train))

    batch_size1 = len(X_train)
    
    training_data_batch = mx.gluon.data.DataLoader(gluon.data.ArrayDataset(nd.array(X_train) , nd.array(y_train)) ,
                                               batch_size=batch_size1 , shuffle= False)
    
    return training_data_batch

In [106]:
x = 10

y = 20

def addd():
    
    return x - y



In [107]:
model.collect_params().initialize(mx.init.Xavier() , ctx= context)
    
trainer = gluon.Trainer(model.collect_params(), 'Adam', {'learning_rate': 0.05})
    
L = gluon.loss.L2Loss()
    
#batch_size1 = len(X_train)
#batch_size_test = len(X_test)

batch_size1 = 5000
batch_size_test = 5000

training_data_batch = mx.gluon.data.DataLoader(gluon.data.ArrayDataset(nd.array(X_train) , nd.array(y_train)) ,
                                               batch_size=batch_size1 , shuffle= False)
validation_data_batches = mx.gluon.data.DataLoader(gluon.data.ArrayDataset(nd.array(X_test) , nd.array(y_test)) ,
                                                batch_size=batch_size_test , shuffle= False)


In [108]:
training_mse = []
validation_mse = []
output_res = []



def model_train(model , epochs ):
    

    
    count = 0    
    
    
    
    for epoch in range(epochs):
        
        print("Epoch" , epoch)
        
        for i, (data,label) in enumerate(training_data_batch):
            
            data = data.as_in_context(context).reshape((-1, lot_size, (maxs - mins)))
            
            
            
            label = label.as_in_context(context).reshape((-1, 1, (maxs - mins)))
            
            
            #print("lbabbeele" , data)
            
            
            with autograd.record():
                
                #print("ddataatata" , data)
                #print("here" , epoch)
                output = model(data)
                
             
                
                loss = L(output, label)
                
                #print(epoch)
                
                if epoch == (epochs-1):
                    
                    #print("here")
                    #print(output)
                    output_res.append(output.asnumpy())
                    #print("here2")
                    #print(output_res)
            
            loss.backward()
            
            
                
            trainer.step(batch_size1 , ignore_stale_grad= True)
                
        training_mse.append(evaluate_accuracy(training_data_batch, model, L))
                
                
        validation_mse.append(evaluate_accuracy(validation_data_batches, model, L))
        
        current_val = evaluate_accuracy(validation_data_batches, model, L)
        
        minimum = np.min(validation_mse)
        print("PRE values >> " , current_val , minimum)
        if (current_val > minimum) & (epoch > (epochs/2) ):
            
            count += 1
            print("Warning!!")
            print(current_val , minimum)
            if (count > 6) & (current_val < 2.5):
                
                output_res.append(output.asnumpy())
                print(current_val , minimum)
                break
                
        if(current_val < 2.15):
            
            output_res.append(output.asnumpy())
            print(current_val , minimum)
            break  

In [109]:
model_train(model , 450  )

Epoch 0
PRE values >>  1539.8493041992188 1539.8493041992188
Epoch 1
PRE values >>  1556.4680480957031 1539.8493041992188
Epoch 2
PRE values >>  2518.8135375976562 1539.8493041992188
Epoch 3
PRE values >>  1312.5809020996094 1312.5809020996094
Epoch 4
PRE values >>  1493.1645202636719 1312.5809020996094
Epoch 5
PRE values >>  715.7873687744141 715.7873687744141
Epoch 6
PRE values >>  459.6250686645508 459.6250686645508
Epoch 7
PRE values >>  610.3267364501953 459.6250686645508
Epoch 8
PRE values >>  576.4760589599609 459.6250686645508
Epoch 9
PRE values >>  492.26646423339844 459.6250686645508
Epoch 10
PRE values >>  472.3459701538086 459.6250686645508
Epoch 11
PRE values >>  505.63948822021484 459.6250686645508
Epoch 12
PRE values >>  465.8997268676758 459.6250686645508
Epoch 13
PRE values >>  436.8503723144531 436.8503723144531
Epoch 14
PRE values >>  438.71150970458984 436.8503723144531
Epoch 15
PRE values >>  441.75850677490234 436.8503723144531
Epoch 16
PRE values >>  437.78903961

PRE values >>  434.98136138916016 434.9745330810547
Epoch 136
PRE values >>  434.98099517822266 434.9745330810547
Epoch 137
PRE values >>  434.98059844970703 434.9745330810547
Epoch 138
PRE values >>  434.9801788330078 434.9745330810547
Epoch 139
PRE values >>  434.9797134399414 434.9745330810547
Epoch 140
PRE values >>  434.979248046875 434.9745330810547
Epoch 141
PRE values >>  434.9787139892578 434.9745330810547
Epoch 142
PRE values >>  434.97815704345703 434.9745330810547
Epoch 143
PRE values >>  434.97753143310547 434.9745330810547
Epoch 144
PRE values >>  434.9768829345703 434.9745330810547
Epoch 145
PRE values >>  434.9761505126953 434.9745330810547
Epoch 146
PRE values >>  434.97537994384766 434.9745330810547
Epoch 147
PRE values >>  434.97454833984375 434.9745330810547
Epoch 148
PRE values >>  434.97364807128906 434.97364807128906
Epoch 149
PRE values >>  434.9726791381836 434.9726791381836
Epoch 150
PRE values >>  434.97161865234375 434.97161865234375
Epoch 151
PRE values >> 

PRE values >>  435.3739242553711 153.42932510375977
Warning!!
435.3739242553711 153.42932510375977
Epoch 251
PRE values >>  435.67496490478516 153.42932510375977
Warning!!
435.67496490478516 153.42932510375977
Epoch 252
PRE values >>  435.9755096435547 153.42932510375977
Warning!!
435.9755096435547 153.42932510375977
Epoch 253
PRE values >>  435.6144256591797 153.42932510375977
Warning!!
435.6144256591797 153.42932510375977
Epoch 254
PRE values >>  435.49874114990234 153.42932510375977
Warning!!
435.49874114990234 153.42932510375977
Epoch 255
PRE values >>  435.1258316040039 153.42932510375977
Warning!!
435.1258316040039 153.42932510375977
Epoch 256
PRE values >>  435.06310272216797 153.42932510375977
Warning!!
435.06310272216797 153.42932510375977
Epoch 257
PRE values >>  434.9842529296875 153.42932510375977
Warning!!
434.9842529296875 153.42932510375977
Epoch 258
PRE values >>  434.98435974121094 153.42932510375977
Warning!!
434.98435974121094 153.42932510375977
Epoch 259
PRE values 

PRE values >>  434.98822021484375 153.42932510375977
Warning!!
434.98822021484375 153.42932510375977
Epoch 326
PRE values >>  434.98822021484375 153.42932510375977
Warning!!
434.98822021484375 153.42932510375977
Epoch 327
PRE values >>  434.98828125 153.42932510375977
Warning!!
434.98828125 153.42932510375977
Epoch 328
PRE values >>  434.98826599121094 153.42932510375977
Warning!!
434.98826599121094 153.42932510375977
Epoch 329
PRE values >>  434.9883270263672 153.42932510375977
Warning!!
434.9883270263672 153.42932510375977
Epoch 330
PRE values >>  434.9883117675781 153.42932510375977
Warning!!
434.9883117675781 153.42932510375977
Epoch 331
PRE values >>  434.98838806152344 153.42932510375977
Warning!!
434.98838806152344 153.42932510375977
Epoch 332
PRE values >>  434.9883728027344 153.42932510375977
Warning!!
434.9883728027344 153.42932510375977
Epoch 333
PRE values >>  434.9884567260742 153.42932510375977
Warning!!
434.9884567260742 153.42932510375977
Epoch 334
PRE values >>  434.98

PRE values >>  435.2694549560547 153.42932510375977
Warning!!
435.2694549560547 153.42932510375977
Epoch 401
PRE values >>  435.5647277832031 153.42932510375977
Warning!!
435.5647277832031 153.42932510375977
Epoch 402
PRE values >>  435.4295959472656 153.42932510375977
Warning!!
435.4295959472656 153.42932510375977
Epoch 403
PRE values >>  435.8014678955078 153.42932510375977
Warning!!
435.8014678955078 153.42932510375977
Epoch 404
PRE values >>  435.6612014770508 153.42932510375977
Warning!!
435.6612014770508 153.42932510375977
Epoch 405
PRE values >>  436.12512969970703 153.42932510375977
Warning!!
436.12512969970703 153.42932510375977
Epoch 406
PRE values >>  435.98973083496094 153.42932510375977
Warning!!
435.98973083496094 153.42932510375977
Epoch 407
PRE values >>  436.5631332397461 153.42932510375977
Warning!!
436.5631332397461 153.42932510375977
Epoch 408
PRE values >>  436.44937896728516 153.42932510375977
Warning!!
436.44937896728516 153.42932510375977
Epoch 409
PRE values >>

In [110]:
model

Sequential(
  (0): LSTM(1 -> 5, TNC, bidirectional)
  (1): LSTM(10 -> 3, TNC, bidirectional)
  (2): LSTM(6 -> 1, TNC, bidirectional)
  (3): Dense(2 -> 656, Activation(relu))
  (4): Dense(656 -> 328, Activation(relu))
  (5): Dense(328 -> 164, Activation(relu))
  (6): Dense(164 -> 82, Activation(relu))
  (7): Dense(82 -> 41, Activation(relu))
  (8): Dense(41 -> 1, linear)
)

In [111]:
np.array(output_res).reshape(-1,)

array([array([[49.193054],
       [49.193054],
       [49.193054],
       ...,
       [49.193054],
       [49.193054],
       [49.193054]], dtype=float32),
       array([[49.386566],
       [49.386566],
       [49.386566],
       ...,
       [49.386566],
       [49.386566],
       [49.386566]], dtype=float32),
       array([[49.389057],
       [49.389057],
       [49.389057],
       ...,
       [49.389057],
       [49.389057],
       [49.389057]], dtype=float32),
       array([[49.233566],
       [49.233566],
       [49.233566],
       ...,
       [49.233566],
       [49.233566],
       [49.233566]], dtype=float32)], dtype=object)

In [48]:
#sc.inverse_transform(np.array(output_res).reshape(-1,1))

In [49]:
y_train

array([[49.],
       [52.],
       [19.],
       ...,
       [38.],
       [83.],
       [46.]])

In [ ]:
validation_mse

In [ ]:
type(output_res)

In [ ]:
%matplotlib inline
plt.figure(figsize = (70 ,30))
plt.plot(y_train[:, 0:1], color = 'red', label = 'Feature A - Actual')
plt.plot(np.array(output_res).reshape(-1,), color = 'blue', label = 'Predicted ')
plt.title('Number - A')
plt.xlabel('Time')
plt.ylabel('Number - A')
plt.legend()
plt.show()

In [50]:
output_test = model(nd.array(X_test))

In [51]:
output_test = output_test.asnumpy()

In [52]:
output_test

array([[48.888077],
       [54.86085 ],
       [17.530804],
       ...,
       [39.02611 ],
       [81.5196  ],
       [64.073784]], dtype=float32)

In [53]:
y_test

array([[49.],
       [52.],
       [19.],
       ...,
       [38.],
       [83.],
       [46.]])

In [ ]:
np.mean(abs(y_test - output_test))

In [ ]:
%matplotlib inline
plt.figure(figsize = (20 ,10))
plt.plot(y_test[:, 0:1], color = 'red', label = 'Feature A - Actual')
plt.plot(output_test[:, 0:1], color = 'blue', label = 'Predicted ')
plt.title('Number - A')
plt.xlabel('Time')
plt.ylabel('Number - A')
plt.legend()
plt.show()

In [54]:
test = y_test

limit = 6

for i in range(1 , limit):
    
    
    test = model(nd.array(test).reshape(-1,1,1))
    
    test = np.around(test.asnumpy())
    
    new = test[-1]
    
    print(new)

[73.]
[67.]
[68.]
[68.]
[68.]


In [ ]:
38 - 21

In [ ]:
38 - 9

In [ ]:
test = df_train.Output

limit = 6

for i in range(1 , limit):
    
    
    test = model(nd.array(test).reshape(-1,1,1))
    
    test = np.around(test.asnumpy())
    
    new = test[-1]
    
    print(new)

In [ ]:
y_test.shape

In [ ]:
y_test

In [ ]:
y_test_for

In [ ]:
y_test_for.shape

In [55]:
y_test_for = y_test.reshape(-1,1,1)

In [56]:
start = 100

In [57]:
start

100

In [58]:
y_test_for[start:len(y_test_for) , :]

array([[[62.]],

       [[ 0.]],

       [[32.]],

       ...,

       [[38.]],

       [[83.]],

       [[46.]]])

In [59]:
len(y_test_for[start:len(y_test_for) , :])

16200

# Iterative incremental Learning (Transfer) for predicting forward Price (Future Price)

In [60]:



limit = 3
k = []

for i in range(1 , limit):
    
    
    

    if i == 1:
        
            

    
        print('1----------------------------')
    
        #output1 =  model( nd.array(X_test_for[start:len(X_test_for) , :]))
        
        output1 =  model( nd.array(y_test_for))
        
        #print(output1)
    
        file_name = "net.params"
        model.save_parameters(file_name)
        
        new = output1[-1].reshape((-1, 1, (maxs-mins)))
        
        new = new.asnumpy()
    
        y_test_for = np.append(y_test_for , new).reshape(-1 , 1 ,(maxs-mins))
    
        #print(X_test_for)
    
        y_test_for = np.delete(y_test_for , 0 ,0)   
    
        k.append(new)
        
    print('other model' , i)
    
    
    
    
    #output1 = output1.asnumpy()
    
    

    print('2------------------------')
    
    #print(X_test_for)
    
    print('3------------------------')
    
        
    #print("data beffore correction")
    #print(y_test_for[start:len(y_test_for) , :])

    

    
    training_data_batch = dataprep(y_test_for[start:len(y_test_for) , :])
    


    model22 = build_net(gluon.nn.Sequential())
    model22.load_parameters(file_name, ctx=context)
    
    trainer = gluon.Trainer(model22.collect_params(), 'adam', {'learning_rate': 0.05})
    #0.05
    
    model_train(model22 , 20)
    
    output1 = model22(nd.array(y_test_for[start:len(y_test_for) , :]))
    
    #print("output1" , output1)
    
    file_name = "net.params"
    model22.save_parameters(file_name)
        

    new = output1[-1].reshape((-1, 1, (maxs-mins)))
    
    new = new.asnumpy()
    
    y_test_for = np.append(y_test_for , new).reshape(-1 , 1 ,(maxs-mins))
    
    
    
    y_test_for = np.delete(y_test_for , 0 ,0)   
    
    
    #print("data after correction")
    #print(y_test_for[start:len(y_test_for) , :])

       
    k.append(new)
    
    

1----------------------------
other model 1
2------------------------
3------------------------
Y>>>>>>>>>>>>>>>>>>
X>>>>>>>>>>>>>>>>>>
Epoch 0
PRE values >>  1157.9508056640625 1.9697749018669128
Epoch 1
PRE values >>  294.68446350097656 1.9697749018669128
Epoch 2
PRE values >>  1072.5162048339844 1.9697749018669128
Epoch 3
PRE values >>  132.21076202392578 1.9697749018669128
Epoch 4
PRE values >>  238.5454444885254 1.9697749018669128
Epoch 5
PRE values >>  455.21319580078125 1.9697749018669128
Epoch 6
PRE values >>  536.8906555175781 1.9697749018669128
Epoch 7
PRE values >>  512.6889724731445 1.9697749018669128
Epoch 8
PRE values >>  366.23533630371094 1.9697749018669128
Epoch 9
PRE values >>  201.0381317138672 1.9697749018669128
Epoch 10
PRE values >>  84.91268157958984 1.9697749018669128
Epoch 11
PRE values >>  194.26603317260742 1.9697749018669128
Warning!!
194.26603317260742 1.9697749018669128
Epoch 12
PRE values >>  257.7402114868164 1.9697749018669128
Warning!!
257.740211486816

In [101]:
k

[array([[[62.898586]]], dtype=float32),
 array([[[45.63886]]], dtype=float32),
 array([[[79.88687]]], dtype=float32)]

In [136]:
k

[array([[[99.13501]]], dtype=float32),
 array([[[79.69946]]], dtype=float32),
 array([[[85.31404]]], dtype=float32)]

In [71]:
k

[array([[[49.173843]]], dtype=float32),
 array([[[41.66715]]], dtype=float32),
 array([[[13.9513035]]], dtype=float32)]

In [61]:
k

[array([[[72.621124]]], dtype=float32),
 array([[[40.970078]]], dtype=float32),
 array([[[32.418823]]], dtype=float32)]

In [ ]:
-
-
-
X
63
32
73
0
76
16
56
83
15
96


In [ ]:
[array([[[3.5939794]]], dtype=float32),
 array([[[61.26958]]], dtype=float32),
 array([[[48.243618]]], dtype=float32),
 array([[[13.0139675]]], dtype=float32),
 array([[[69.53148]]], dtype=float32),
 array([[[26.545092]]], dtype=float32)]

In [ ]:


# limit = 7
# k = []

# for i in range(1 , limit):
    
    
    

#     if i == 1:
        
            

    
#         print('1----------------------------')
    
#         #output1 =  model( nd.array(X_test_for[start:len(X_test_for) , :]))
        
#         output1 =  model( nd.array(X_test_for))
        
#         #print(output1)
    
#         file_name = "net.params"
#         model.save_parameters(file_name)
        
#         new = output1[-1].reshape((-1, 1, (maxs-mins)))
        
#         new = new.asnumpy()
    
#         X_test_for = np.append(X_test_for , new).reshape(-1 , 1 ,(maxs-mins))
    
#         #print(X_test_for)
    
#         X_test_for = np.delete(X_test_for , 0 ,0)   
    
        
        
#     print('other model' , i)
    
    
    
    
#     #output1 = output1.asnumpy()
    
    

#     print('2------------------------')
    
#     #print(X_test_for)
    
#     print('3------------------------')
    
        
#     print("data beffore correction")
#     print(X_test_for[start:len(X_test_for) , :])

    
#     #print(X_test_for[start:len(X_test_for) , :])
    
#     training_data_batch = dataprep(X_test_for[start:len(X_test_for) , :])
    
#     #training_data_batch = dataprep(X_test_for)
    
#     print(type(X_test_for))
    
#     #training_data_batch = dataprep(X_test_for)
    
#     #print(X_test_for)
#     #print(training_data_batch)

#     model22 = build_net(gluon.nn.Sequential())
#     model22.load_parameters(file_name, ctx=context)
    
#     trainer = gluon.Trainer(model22.collect_params(), 'adam', {'learning_rate': 0.05})
    
    
#     model_train(model22 , 10)
    
#     output1 = model22(nd.array(X_test_for[start:len(X_test_for) , :]))
    
#     print("output1" , output1)
    
#     file_name = "net.params"
#     model22.save_parameters(file_name)
        

#     new = output1[-1].reshape((-1, 1, (maxs-mins)))
    
#     new = new.asnumpy()
    
#     X_test_for = np.append(X_test_for , new).reshape(-1 , 1 ,(maxs-mins))
    
#     #print(X_test_for)
    
#     X_test_for = np.delete(X_test_for , 0 ,0)   
    
    
#     print("data after correction")
#     print(X_test_for[start:len(X_test_for) , :])

       
#     k.append(new)

In [ ]:
k

In [ ]:
np.array(k).reshape(-1  , (maxs-mins))[: , 0:1]

In [ ]:
%matplotlib inline
plt.figure(figsize = (30 ,20))
plt.plot(y_test[:, 0:1], color = 'red', label = 'Feature A - Actual')
plt.plot(np.array(k).reshape(-1  , (maxs-mins))[: , 0:1], color = 'blue', label = 'Predicted ')
plt.title('Number - A')
plt.xlabel('Time')
plt.ylabel('Number - A')
plt.legend()
plt.show()

In [ ]:

num_hidden = 256
num_outputs = 1
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

In [ ]:
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=context)

In [ ]:
net(nd.ones((1, 100), ctx=context))

In [ ]:
nd.ones((1, 100))

In [ ]:
filename = "testnet.params"
net.save_parameters(filename)
net2 = gluon.nn.Sequential()
with net2.name_scope():
    net2.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net2.add(gluon.nn.Dense(num_hidden, activation="relu"))
    
    net2.add(gluon.nn.Dense(num_outputs))

In [ ]:
net2.load_parameters(filename, ctx=context)
net2(nd.ones((1, 100), ctx=context))

In [ ]:
context

In [ ]:
constants